In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants

from lmfit.models import ExpressionModel

import arviz as az

RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.library["arviz-darkgrid"]['figure.constrained_layout.use'] = False
az.style.use("arviz-darkgrid")

%matplotlib inline

In [ ]:
abs0 = 273.15
A = 2 * constants.h * (constants.c)**2
B = (constants.h * constants.c) / constants.k

## Basic wavelength dependence of intensity using Planck law

$$
B(\lambda, T) = \frac{2hc^2}{\lambda^5}\frac{1}{\exp{\frac{hc}{\lambda k_B T}}-1} \\

I = \epsilon(\lambda) B \qquad (0 < \epsilon < 1)
$$

In [ ]:
model_planck = ExpressionModel(
    'A / (lam**5 * (exp(B / (lam * T)) - 1))', 
    independent_vars=['lam']
)

In [ ]:
fig = plt.figure()
ax = plt.gca()

lam_range = (0.3e-6, 6e-6)

lam = np.linspace(*lam_range, 101)
for T in [100, 200, 400, 500, 750, 1000, 2000, 3000]:
    params = model_planck.make_params(A=A, B=B, T=T+abs0)
    I = model_planck.eval(params, lam=lam)
    ax.plot(lam, I, label=T)

# ax.set_yscale('log') 
ax.set_ylim((0, 1e11))
ax.set_xlim(lam_range)
ax.set_xlabel('Wavelength ($\mu$m)')
ax.set_ylabel('Intensity')
ax.legend(title='Temp ($^\circ$C)', ncol=3)

# for cam_name, cam_spec in camera_specs.items():
#     ax.axvspan(cam_spec['lam_0'], cam_spec['lam_inf'], 
#                color=cam_spec['colour'], alpha=0.5, label=cam_name)


In [ ]:
model_quadratic = ExpressionModel(
    'A * (a0 + a1*lam + a2*lam**2) / (lam**5 * (exp(B / (lam * T)) - 1))', 
    independent_vars=['lam']
)

In [ ]:
fig = plt.figure()
ax = plt.gca()

lam = np.linspace(0.1e-6, 10e-6, 101)

T = 1000

a0, a1, a2 = 1, 0, 0
params = model_quadratic.make_params(A=A, B=B, T=T+abs0, a0=a0, a1=a1, a2=a2)
I = model_quadratic.eval(params, lam=lam)
ax.plot(lam*1e6, I, label='black body')

a0, a1, a2 = 0.5, 0, 0
params = model_quadratic.make_params(A=A, B=B, T=T+abs0, a0=a0, a1=a1, a2=a2)
I = model_quadratic.eval(params, lam=lam)
ax.plot(lam*1e6, I, label='grey body ($\epsilon = 0.5$)')

# ax.set_yscale('log') 
# ax.set_ylim((1e6, 1e13))
ax.set_xlabel('Wavelength ($\mu$m)')
ax.set_ylabel('Intensity')
ax.legend()

plt.show()

## Create test data
Linear temperature gradient along a line of a grey body with noise

In [ ]:
def intensity_func(T, lam, epsilon):
    A = 2 * constants.h * (constants.c)**2
    B = (constants.h * constants.c) / constants.k
    
    A * epsilon / (lam**5 * (np.exp(B / (lam * T)) - 1))

In [ ]:
T_test = np.linspace(500+abs0, 3000+abs0, 1000)
lam_test = np.linspace(0.1e-6, 10e-6, 10)
epsilon = 0.8
noise_sigma = 1e7

mod_grey = ExpressionModel('A * epsilon / (lam**5 * (exp(B / (lam * T)) - 1))', independent_vars=['lam', 'T'])

params = mod_grey.make_params(A=A, B=B, epsilon=epsilon)
grid = np.meshgrid(lam_test, T_test, indexing='ij')
I_test = mod_grey.eval(params, lam=grid[0], T=grid[1]) + np.random.normal(0, noise_sigma, (lam_test.size, T_test.size))
I_test[I_test < 0] = 0.

In [ ]:
i_T = 999

lam = np.linspace(0.1e-6, 10e-6, 100)

mod_grey = ExpressionModel('A * epsilon / (lam**5 * (exp(B / (lam * T)) - 1))', independent_vars=['lam'])
params = mod_grey.make_params(A=A, B=B, epsilon=epsilon, T=T_test[i_T])
I = mod_grey.eval(params, lam=lam)

plt.figure(figsize=(4, 3))
ax = plt.gca()

ax.plot(lam*1e6, I)
ax.scatter(lam_test*1e6, I_test[:, i_T], marker='x')

ax.set_title(f'{T_test[i_T]-abs0:.0f} $^\circ$C')
ax.set_xlabel('Wavelength ($\mu$m)')
ax.set_ylabel('Intensity')


## And fit to model
Linear temperature gradient along a line of a grey body with noise

In [ ]:
mod_grey = ExpressionModel('A * epsilon / (lam**5 * (exp(B / (lam * T)) - 1))', independent_vars=['lam'])
params = mod_grey.make_params(
    A={'value': A, 'vary': False},
    B={'value': B, 'vary': False},
    epsilon={'value': 0.5, 'min': 0, 'max': 1},
    T={'value': 1500+abs0, 'min': 300, 'max': 5300},
)

T_pred = np.zeros_like(T_test)
epsilon_pred = np.zeros_like(T_test)
pred_covar = np.zeros((len(T_test), 2, 2))

for i_T in range(len(T_test)):
    result = mod_grey.fit(I_test[:, i_T], params, lam=lam_test)
    T_pred[i_T] = result.best_values['T']
    epsilon_pred[i_T] = result.best_values['epsilon']
    pred_covar[i_T] = result.covar


In [ ]:
plt.figure()
plt.plot(T_pred-abs0)

plt.figure()
plt.plot(epsilon_pred)

In [ ]:
std = 2*np.sqrt(pred_covar[:, 1, 1])
plt.figure()
plt.fill_between(np.arange(len(pred_covar)), std, -std, 
                 color='gray', alpha=0.5)
plt.plot(T_pred - T_test)

std = np.sqrt(pred_covar[:, 0, 0])
plt.figure()
plt.fill_between(np.arange(len(pred_covar)), std, -std, 
                 color='gray', alpha=0.5)
plt.plot(epsilon - epsilon_pred)

In [ ]:
result